# Salary Predictions Based on Job Descriptions

# <span style="color:green">Part 1 - DEFINE</span>

## ---- Define the problem ----

This project intends to predict salaries for given job descriptions.

Job descriptions have eight __features__:

* __jobId__ = a unique index for each job

* __companyId__ = a categorical ID for the company the job is for

* __jobType__ = a categorical feature describing the role

* __degree__ = a categorical feature describing the required education level

* __major__ = a categorical feature conveying the field in which a degree is required, if any

* __industry__ = a categorical feature describing the industry to which the job belongs

* __yearsExperience__ = a numerical feature measuring how many years of work experience are required for the role

* __milesFromMetropolis__ = a numerical feature measuring how far the workplace is located from a metropolis


The __target__ is __salary__ (in 1000 USD). Salaries are given in the training set and need to be predicted for the test set.

In [1]:
# Third party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor

# Local imports
from eda.stats import interquartile_rule
from eda import plot
from feature_engineering import encoders
from exceptions import NotUniqueError

# Author information
__author__ = "Paawan Sharma"
__email__ = "paawansharma@protonmail.com"

%matplotlib inline

The following cell prevents jupyter from creating scrollable subframes for plots, instead showing the entire set of generated plots without the need for scrolling.

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# <span style="color:green">Part 2 - DISCOVER</span>

## ---- Load the data ----

In [3]:
# Load the training and test data in pandas DataFrames

train_data = pd.read_csv("../data/train_features.csv", header=0)
train_data["salary"] = pd.read_csv("../data/train_salaries.csv", header=0)["salary"]

test_data = pd.read_csv("../data/test_features.csv", header=0)

### Take an initial look at the data.

In [4]:
# display(train_data)
# print("="*98)
# train_data.info()

In [5]:
# display(test_data)
# print("="*88)
# test_data.info()

## ---- Clean the data ----

### Look for duplicate data in the training set. Duplicate job IDs may indicate corrupt data.

In [6]:
# print(
#     "There are {} duplicate rows in the training set.".format(
#         train_data.duplicated().sum()
#     )
# )
# print(
#     "There are {} duplicate jobIDs in the training set.".format(
#         train_data["jobId"].duplicated().sum()
#     )
# )

### Look for invalid data.

We know from earlier that there are no null values. We can check that all values are appropriate for each variable.

In [7]:
# # Numerical features in both dataframes

# for df_name, df in {"test set": test_data, "training set": train_data}.items():
#     print("Checking {}".format(df_name))
#     print("Are all years of experience non-negative?")
#     print(df[df["yearsExperience"] < 0].empty)
#     print("Are all miles from metropolis non-negative?")
#     print(df[df["milesFromMetropolis"] < 0].empty)
#     print("\n")

# print("Are all salary values in training set positive?")
# print(train_data[train_data["salary"] <= 0].empty)

### Observe jobs whose salaries are not positive

In [8]:
# train_data[train_data["salary"] <= 0]

None of these salaries is negative. A salary can be zero (or rounded down to zero) if the employee opts to receive alternative compensation so these data may be valid. However, it is also possible these are missing values. Since the training dataset is large (1,000,000 samples), it is safe to drop these samples.

In [9]:
train_data.drop(train_data[train_data["salary"] == 0].index, inplace=True)
print("The number of samples is now {:,}.".format(train_data.shape[0]))

The number of samples is now 999,995.


In [10]:
# # Categorical features in both dataframes

# for df_name, df in {"training set": train_data, "test set": test_data}.items():
#     print("Checking {}\n".format(df_name))
#     for feature in ["jobType", "degree", "major", "industry"]:
#         print("Values for {} are: {}\n".format(feature, list(df[feature].unique())))
#     print("\n")

All values appear to be reasonable.

### Check that majors are only given for jobs that require degrees

In [11]:
# list(train_data[train_data["major"] != "NONE"].degree.unique())

### As jobIDs are unique, we can set them as the indices for the dataframes.

In [12]:
for df in [train_data, test_data]:
    df.set_index("jobId", inplace=True)

### Finally, check that none of the training jobIds are repeated in the test set.

In [13]:
# print(
#     "The intersection contains {} samples.".format(
#         train_data.index.intersection(test_data.index).size
#     )
# )

## ---- Explore the data (EDA) ----

### Start by getting a description of all categorical and numerical variables.

In [14]:
# train_data.describe(include=["O"])

It is noteworthy that there are 63 different companies in the dataset. The other categorical features have few unique values and therefore will probably be more useful to models.

In [15]:
# train_data.describe(include=np.number)

All numerical ranges seem reasonable.

### Explore the distribution of the target (salary).

In [16]:
sns.set_theme(style="whitegrid", palette="colorblind")
sns.set_context("poster")

# plot.plot_target("salary", train_data, target_label="Salary / 1000 USD")

The salary is approximately normally distributed. There are some outliers on the upper end of the distribution. These can be explored using the interquartile rule for outliers.

In [17]:
# salary_outliers, salary_uppers, salary_lowers = interquartile_rule("salary", train_data)

# print("There are {} lower outliers.".format(salary_lowers))

# for feature in ["jobType", "degree", "industry"]:
#     display(salary_outliers[feature].value_counts())

The bulk of the outliers, which are all upper outliers, correspond to typically high-paying roles, high educational qualifications and profitable industries (such as oil and finance). This is to be expected and provides confidence in the validity of the data.

It is worthwhile to explore those outliers which require no educational qualification. There are 208 such outliers.

In [18]:
# salary_outliers[salary_outliers.degree == "NONE"].jobType.value_counts()

These are all typically high-paying roles, which is a good indication.

### Investigate counts and correlations with salary for each categorical feature

In [19]:
categoricals = list(train_data.columns[train_data.dtypes == "object"])
numericals = list(train_data.columns[train_data.dtypes == "int64"])[:-1]

# for feature in categoricals:
#     plot.plot_categorical(feature, "salary", train_data)

__companyId__ has no association with salary. It is worth investigating whether the number of dataset samples belonging to a given company has any relationship with salary as this might convey some information about the size of the company and hence perhaps the salaries it pays.

__jobType__ shows a clear association with salary. Chief roles are the highest paying and janitor the lowest.

__degree__ shows a clear difference in salaries between those without a degree and those with one. Beyond this, the correlation is slight.

__major__ shows a very slight association, with engineering being the major corresponding to the highest median salary.

__industry__ shows a clear association, with oil and finance having the highest median salaries.

All correlations appear to be linear.

In [20]:
# plot.categorical_correlation(
#     feature="companyId",
#     target="salary",
#     dataframe=train_data,
#     groupfunc=np.size,
#     x_label="Number of entries for company in training data",
#     y_label="Mean salary / 1000 USD",
# )

There is a _very_ slight positive correlation between the number of jobs the company has in the training data and the mean salary for jobs in that company.

### Investigate associations of numerical features with salary

In [21]:
# for feature in numericals:
#     plot.plot_numerical(feature, "salary", train_data, target_unit="1000 USD")

Both numerical featuress' counts have approximately uniform distributions.

__yearsExperience__ is positively correlated with salary. This is to be expected, since career progression over time usually results in higher pay.

__milesFromMetropolis__ is negatively correlated with salary. Again, this is to be expected: jobs in cities tend to offer higher salaries to account for the generally higher costs of living.

### Look for correlations between features

Categorical features need to be encoded before the relationships they have with other features and the target can be explored.

There are multiple options for encoding:

1. __dummy coding__: one-hot encoding with one dummy variable dropped

2. __ordinal encoding on ordered categories__: order categories by some metric (such as mean salary) then encode with integers
 * Metrics to consider: mean, median

3. __label encoding__: encode categories with some metric (such as mean salary)
 * Metrics to consider: mean, median
 
Although each of these encodings (particularly the latter two) will convey similar information about the dataset, it is worthwhile to see the heatmaps they each produce as we shall need to use encoding later on for feature engineering. Therefore, we shall explore each of these encodings in turn here.

#### Dummy coding

For dummy coding, we will exclude companyId from our heatmap as it has a very weak association with salary and because the large number of dummy variables would render the heatmap intractable.

Also note that one dummy variable has been dropped from each category so as not to introduce collinearity. Therefore, for instance, the heatmap will not contain a column or row for jobType=CEO (the first level in the alphabetically sorted column of jobTypes).

A clustermap was generated in order to get a new index ordering that would more clearly display the clusters of correlated variables. The new order is passed to the correlation_map method of our encoder.

In [22]:
vlag = sns.color_palette("vlag", as_cmap=True)

In [23]:
# reordering = [
#     11,
#     5,
#     13,
#     20,
#     22,
#     1,
#     6,
#     8,
#     26,
#     27,
#     7,
#     24,
#     25,
#     3,
#     23,
#     4,
#     0,
#     2,
#     9,
#     18,
#     17,
#     16,
#     12,
#     21,
#     15,
#     19,
#     10,
#     14,
# ]

# dummy_encoder = encoders.Dummy_Encoder(exclude=['companyId'])
# dummy_encoder.fit(train_data)
# dummy_encoder.correlation_map(train_data, vlag, 175, 7.5, 1, reordering=reordering)

__Observations of interest__


* __Salary__ is __positively correlated__ with

 * yearsExperience (the most positive correlation),
 
 * degree_DOCTORAL, degree_MASTERS,
 
 * jobType_CFO, jobType_CTO, jobtype_VICE_PRESIDENT
 
 * industry_OIL, industry_FINANCE, industry_WEB,
 
 * and all dummy variables indicating an existent major, with major_ENGINEERING having the highest correlation and major_LITERATURE the lowest.
 
* __Salary__ is __negatively correlated__ with

 * jobType_JANITOR (the highest magnitude correlation), jobType_JUNIOR, jobType_MANAGER
 ,
 * major_NONE,
 
 * milesFromMetropolis,
 
 * degree_NONE, degree_HIGH_SCHOOL,
 
 * and industry_EDUCATION, industry_SERVICE, industry_HEALTH.

* major_NONE is strongly positively correlated with degree_HIGH_SCHOOL and degree_NONE (as noted earlier) and with jobType_JANITOR.

* As expected, degree_HIGH_SCHOOL and degree_NONE are positively correlated with jobType_JANITOR.

#### Ordinal encoding on ordered categories

For ordinal encoding, we will consider ordering both by mean salary and by median salary.

In [24]:
# for average in [np.mean, np.median]:
#     print("\n".join([average.__name__.title(), "=" * len(average.__name__)]))
#     try:
#         ordinal_encoder = encoders.Ordinal_Encoder(average, 'salary')
#         ordinal_encoder.fit(train_data)
#         ordinal_encoder.correlation_map(train_data, vlag, 100, 10, 2)
#     except NotUniqueError as nue:
#         print(nue)

__Observations__

* Ordinal encoding is much easier to interpret than dummy coding as it doesn't produce a large array.
* We have already seen the associations between salary and individual features but we can now quantify these associations.
* milesFromMetropolis is the only feature negatively correlated with salary (note that categorical features cannot have negative associations with the target due to the way values are ordered for ordinal encoding).
* companyId has a very weak association with salary.
* degree and yearsExperience have the highest postive correlations with salary, followed by major and industry in that order.
* We can see that degree, major and jobType are positively associated with each other under ordinal encoding. The relationship between degree and major is particularly strong.

#### Target encoding

For target encoding, we shall only be using the mean for labelling groups as the median will result in the same error as in ordinal encoding.

In [25]:
# target_encoder = encoders.Target_Encoder(np.mean, 'salary')
# target_encoder.fit(train_data)
# target_encoder.correlation_map(train_data, vlag, 100, 10, 2)

__Observations__

The results are similar to those for ordinal encoding but magnitudes of correlations are generally slightly greater. This is to be expected as target encoding preserves more information about the magnitudes of mean salary values of groups than a mere ordering does.

## ---- Establish a baseline ----

### Accuracy metric

We shall use mean squared error (MSE) as a metric for assessing the accuracy of models.

### Baseline model

Our baseline model will be mean salary for each job type.

In [26]:
true_salaries = train_data["salary"]

# Calculate baseline predictions
mapping = train_data.groupby("jobType")["salary"].mean()
baseline_pred = train_data["jobType"].replace(mapping)

# MSE
baseline_mse = mean_squared_error(true_salaries, baseline_pred)
print("The MSE of the baseline model is {}".format(baseline_mse))

The MSE of the baseline model is 963.9252996562975


## ---- Hypothesise a solution -----

### Models to try

Kinds of model we could try for this problem are:

* multiple linear regression

* support vector regression

* decision tree regression

* random forest regression

* gradient boosting regression


### Feature selection and engineering


* companyId is not associated with salary under any of the encoding schemes we tried, hence we could try dropping it from the data.

* Target encoding would be a good encoding system to try as it doesn't produce a high number of features (and so doesn't require us to drop companyId) and gives good correlations with salary.

* For linear models, we need to scale our features.

* We could engineer pairwise interaction features. These may help with linear models.

# <span style="color:green">Part 3 - DEVELOP</span>

## ---- Engineer features -----

In [27]:
# # from sklearn.ensemble import RandomForestRegressor

# # n_estimators=100
# # rf_regression = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1, verbose=5)

In [28]:
import preprocessing


dummy_encoder = encoders.Dummy_Encoder(exclude=["companyId"])
target_encoder = encoders.Target_Encoder(np.mean, "salary")
ordinal_encoder = encoders.Ordinal_Encoder(np.mean, "salary")

encoders_dict = {
    "Dummy coding": dummy_encoder,
    "Target encoding": target_encoder,
    "Ordinal encoding": ordinal_encoder,
}

In [29]:
dummy_regressors_dict = {}
group_regressors_dict = {}

which_dict = {
    dummy_encoder: dummy_regressors_dict,
    target_encoder: group_regressors_dict,
    ordinal_encoder: group_regressors_dict,
}


dummy_regressors_dict["Linear regression"] = LinearRegression()

for md in [5, 10, 15, 20, 25]:
    group_regressors_dict['DTR maxdepth {}'.format(md)] = DecisionTreeRegressor(max_depth=md)
    dummy_regressors_dict['DTR maxdepth {}'.format(md)] = DecisionTreeRegressor(max_depth=md)

sv_regression = LinearSVR(verbose=True)
scaled_sv_regression = make_pipeline(StandardScaler(), sv_regression, verbose=True)
group_regressors_dict["Scaled SVR"] = scaled_sv_regression


def cv_mse(estimator, X, y):
    return cross_val_score(
        estimator=estimator,
        X=X,
        y=y,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
        verbose=True,
    )


for enc_name, enc_obj in encoders_dict.items():

    X_train, y_train, encoded_test = preprocessing.encode_and_split(
        enc_obj, train_data, test_data, target="salary"
    )

    for regressor_name, regressor in which_dict[enc_obj].items():
        cvs = cv_mse(regressor, X_train, y_train)
        print(
            "{} with {}\n{}\n\n".format(regressor_name, enc_name, (cvs * -1).mean())
        )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Linear regression with Dummy coding
384.414337322381




[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   18.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


DTR maxdepth 5 with Dummy coding
721.8098598289428




[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   22.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

DTR maxdepth 20 with Ordinal encoding
620.6791360991081




[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   26.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


DTR maxdepth 25 with Ordinal encoding
736.5237890196073




[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   42.1s finished


Scaled SVR with Ordinal encoding
398.26120060300957




## ---- Create models -----

## ---- Test models -----

## ---- Select best model -----

# <span style="color:green">Part 4 - DEPLOY</span>

## ---- Automate pipeline -----

## ---- Deploy solution -----

## ---- Measure efficacy -----